# Precificando derivativos através do modelo de volatilidade estocástica de Heston

\begin{align}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} & = 0
\end{align}

```python
import numpy as np
from scipy import inf, pi, exp, linspace, zeros, real, imag, array, log
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import differential_evolution
import pandas as pd
import os
import time
from matplotlib import pyplot
import math
```

In [ ]:
##Primeiros tratamentos dos dados

iv_surface = pd.read_csv("data/spy_options.csv")
iv_surface.rename(columns={'Strike': 'strike',
                           'Implied Volatility': 'iv',
                           'Last Price': "price"}, inplace=True)

iv_surface["moneyness"] = iv_surface["spot"] / iv_surface["strike"]

iv_surface = iv_surface.loc[(iv_surface["tipo"] == "calls") & 
(iv_surface["t_ano"] > 0.1) & 
(iv_surface["t_ano"] < 5) & 
(iv_surface["moneyness"] < 1.66) & 
(iv_surface["moneyness"] > 0.7) & 
(iv_surface["iv"] > 0) &
(iv_surface["strike"].isin([180, 200, 220, 240, 270, 299, 315, 330, 350, 370, 400, 420])), 
["spot", "strike", 'moneyness', "price", "iv", "tipo", "t_ano"]]

# Funções necessárias

Aqui, chamamos as funções necessárias

In [ ]:
def heston_phi(k, tau, *parms):

    ## PARÂMETROS
    ## v0: volatilidade inicial
    ## v_long: média de longo prazo da volatilidade
    ## mean_reversion: velocidade da reversão à média
    ## vol_vol: volatilidade da volatilidade
    ## rho: correlação entre o ativo subjacente e a volatilidade
    ## tau: prazo de expiração
    ## S0: preço inicial do ativo subjacente
    ## K: preço de strike

    v0, v_long, mean_reversion, vol_vol, rho = parms

    b = mean_reversion + 1j*rho*vol_vol*k
    d = np.sqrt( b**2 + (vol_vol**2)*k*(k-1j) )
    g = (b - d)/(b + d)
    T_m = (b - d)/(vol_vol**2)
    T = T_m * ( 1 - exp(-d*tau) )/( 1 - g*exp(-d*tau) )
    W = mean_reversion * v_long * ( tau*T_m - 2*log( ( 1 - g*exp(-d*tau) )/( 1 - g ) )/(vol_vol**2) )

    return exp(W + v0*T)

def heston_phi_transform(tau, x, *parms):
    integrand = lambda k: 2 * real( exp(-1j*k*x) * heston_phi(k + 0.5*1j, tau, *parms) )/(k**2 + 1.0/4.0)
    return quad(integrand, 0, 50)[0]

def heston_call(F, K, tau, *parms):
    '''Heston call'''
    x = log(F/K)
    return F - (np.sqrt(K*F)/(2*pi)) * heston_phi_transform(tau, x, *parms)

def heston_evaluate(chromo):
    """docstring for heston_evaluate"""
    v0 = chromo[0]
    v_long = chromo[1]
    mean_reversion = chromo[2]
    vol_vol = chromo[3]
    rho = chromo[4]

    lista_erro = []    
  
    for index, row in iv_surface.iterrows():
        lista_erro.append((row["price"] - heston_call(row["spot"], row["strike"], row["t_ano"], v0, v_long, mean_reversion, vol_vol, rho))**2)

    diffs = sum(lista_erro)

    return diffs

# Calibração do modelo

Aqui vamos calibrar o modelo

In [ ]:
##Calibrando o modelo via Differential Evolution


result = differential_evolution(heston_evaluate,
    workers = -1,
    disp = True,
    bounds = [(0,1), (0, 1), (0, 5), (0, 5), (-1, 1)],
    maxiter = 30)

optimized_parameters = result.x

price_pred_list = []

for index, row in iv_surface.iterrows():
    price_pred_list.append(heston_ucall(row["spot"], row["strike"], row["t_ano"], *optimized_parameters))